## Including Libraries and Package Installation

In [ ]:
!pip install datasets

In [1]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import os
import time
import json
import torch
def tic():
    return time.time()
def toc(starttime):
    return time.time()-starttime

## English To Persian Translation Model Initialization

In [ ]:
model_size = "large"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-translation_en_fa"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)
PATH="youcookii_annotations_trainval.json"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
# Check if GPU is available and move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 1024)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 1024)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=Fals

## Downloading output.json For Training Data

In [4]:
counter=0
if not os.path.exists(PATH):
    !wget http://youcook2.eecs.umich.edu/static/YouCookII/youcookii_annotations_trainval.tar.gz
    !tar -xvzf youcookii_annotations_trainval.tar.gz
    with open('youcookii_annotations_trainval.json', 'r') as file:
        data = json.load(file)
        data = data['database']
else:
    with open('progress.txt', 'r') as file:
        counter=int(file.read())
    with open('output.json', 'r') as file:
        data = json.load(file)

--2024-12-10 09:15:28--  http://youcook2.eecs.umich.edu/static/YouCookII/youcookii_annotations_trainval.tar.gz
Resolving youcook2.eecs.umich.edu (youcook2.eecs.umich.edu)... 141.212.115.67
Connecting to youcook2.eecs.umich.edu (youcook2.eecs.umich.edu)|141.212.115.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337048 (329K) [application/octet-stream]
Saving to: ‘youcookii_annotations_trainval.tar.gz’

youcookii_annotatio 100%[===================>] 329.15K   394KB/s    in 0.8s    

2024-12-10 09:15:30 (394 KB/s) - ‘youcookii_annotations_trainval.tar.gz’ saved [337048/337048]

youcookii_annotations_trainval.json


In [28]:
sentences = []
for item in data.values():
  annotations = item['annotations']
  for annotation in annotations:
    sentences.append(annotation['sentence'])
for i in range(10):
  print(sentences[i])

spread margarine on two slices of white bread
place a slice of cheese on the bread
place the bread slices on top of each other and place in a hot pan
flip the sandwich over and press down
cut the sandwich in half diagonally
pick the ends off the verdalago
combine lemon juice sumac garlic salt and oil in a bowl
chop lettuce and place it in a bowl
add verdalago pepper cucumbers tomatoes herbs and onions to the lettuce in the bowl
pour the dressing over the salad and mix


## Translation Scripts Modules

In [40]:
def cache(translations):
    with open('Translations.txt', 'w') as file:
      for string in translations:
        file.write(string + "\n")

In [ ]:
def add_translation(trans):
    j = 0
    for key in data.keys():
        item = data[key]
        annotations = item['annotations']
        for i in range(len(annotations)):
            if j == len(trans):
                break
            data[key]['annotations'][i]['translation'] = trans[j]
            j+=1


مارگارین را روی دو تکه نان سفید پهن کنید
یک تکه پنیر روی نان بگذار
تکه های نان را روی هم قرار دهید و در ماهیتابه داغ قرار دهید.
ساندویچ را به طرف پایین پرت کن و فشارش بده
ساندویچ را به طور مورب نصف کنید
نوک درختان را از روی خار بردارید
آب لیموی نمک سیر و روغن را در کاسه ای مخلوط کنید
کاهو را قطعه قطعه کن و در کاسه بگذار
اضافه کردن خیار شور فلفلی، خیار سبز، سبزیجات و پیاز به کاسه کاهو
سس را روی سالاد بریز و مخلوطش کن.
پیتا را به سالاد اضافه کنید و مخلوط کنید.
نمک، شیر و سس داغ را در کاسه ای مخلوط کنید
جوجه را در کاسه کوچکی قرار دهید و روی آن آب شور بریزید و برای یک ساعت سردش کنید.
آب شور را برای جوجه ها بریزید و کاملا آن را بشوئید و آب اضافی را تکان بدهید.
جوجه را در بشقاب یا سینی بگذار و با ادویه مخلوط به طور سخاوتمندانه ای آن را چاشنی بزن.
آرد را با نمک فلفل و نمک کرفس چاشنی بزنید
تکه های مرغ را در آرد بپوشانید
جوجه را در روغن داغ قرار دهید و تا قهوه ای طلایی سرخ کنید.
گوشت های تیره را با گوشت های سفید سرخ کنید
خرده نان و پنیر پارمزان را مخلوط کنید.
جوجه را کوبید
نمک و فلفل روی مرغ ما

In [60]:
def run_model_batch(input_strings, batch_size=128,max_new_tokens=50,initial_index=0, **generator_args):
    translations = []
    counter=initial_index
    start=tic()
    elapsed_time=0
    #trans_check=initial_index
    for i in range(initial_index, len(input_strings), batch_size):
        batch = input_strings[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        # Move input tensors to GPU
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        res = model.generate(input_ids, attention_mask=attention_mask,max_new_tokens=max_new_tokens, **generator_args)
        outputs = tokenizer.batch_decode(res, skip_special_tokens=True)
        translations.extend(outputs)
        if(toc(start)>=60):
          elapsed_time+=toc(start)
          print(f"Number of sentences translated : {len(translations)} , Elapsed time : {elapsed_time} seconds")
          start=tic()



    return translations

In [65]:
trans = run_model_batch(sentences)
cache(trans)

Number of sentences translated : 1792 , Elapsed time : 61.52029037475586 seconds
Number of sentences translated : 3712 , Elapsed time : 123.75262308120728 seconds
Number of sentences translated : 5504 , Elapsed time : 183.85102891921997 seconds
Number of sentences translated : 7296 , Elapsed time : 244.77609515190125 seconds
Number of sentences translated : 9216 , Elapsed time : 309.19386434555054 seconds
Number of sentences translated : 11136 , Elapsed time : 371.63316822052 seconds
Number of sentences translated : 12928 , Elapsed time : 435.378050327301 seconds


In [66]:
print(len(sentences))
print(f"Example : {13828+1}-"+sentences[13828])
for i in range(16):
  print(f"{i+1}-"+sentences[i])

13829
Example : 13829-place the bread on top of the bread
1-spread margarine on two slices of white bread
2-place a slice of cheese on the bread
3-place the bread slices on top of each other and place in a hot pan
4-flip the sandwich over and press down
5-cut the sandwich in half diagonally
6-pick the ends off the verdalago
7-combine lemon juice sumac garlic salt and oil in a bowl
8-chop lettuce and place it in a bowl
9-add verdalago pepper cucumbers tomatoes herbs and onions to the lettuce in the bowl
10-pour the dressing over the salad and mix
11-add the fried pita to the salad and mix
12-combine salt butter milk and hot sauce in a bowl
13-place chicken in a small bowl and pour brine over chicken and freeze it for 1 hour
14-pour off the brine and rinse chicken off thoroughly and shake off excess water
15-place chicken on a plate or tray and season generously with mixed spices
16-season the flour with salt pepper and celery salt


In [67]:
add_translation(trans)
with open('output.json', 'w') as file:
    json.dump(data,file)